In [ ]:
# import all necessary packages for CBOW
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import random
import os
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import collections
import itertools
import re
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from nltk.stem import WordNetLemmatizer
from scipy.stats import pearsonr, spearmanr
from torchtext.vocab import GloVe
from sklearn.model_selection import train_test_split

In [5]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print device name: get_device_name()
# print(torch.cuda.get_device_name(0))

In [2]:
device = torch.device('cpu')

In [6]:
import io
from gensim.models import KeyedVectors
import gensim.downloader as api
from gensim import matutils
from numpy import dot


print(list(api.info()['models'].keys()))

# def load_vectors(fname):
#     fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, fin.readline().split())
#     data = {}
#     for line in fin:
#         tokens = line.rstrip().split(' ')
#         data[tokens[0]] = map(float, tokens[1:])
#     return data

# fasttext_data = load_vectors('/content/drive/My Drive/wiki-news-300d-1M.vec')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [7]:
model = api.load('fasttext-wiki-news-subwords-300')

In [5]:
# Save the model
# torch.save(model, './partA_pth/fasttext_subwords300.pth')

In [8]:
def get_word_embedding(model ,word):
    return model[word]

In [ ]:
# Cosine similarity between same words
w1 = model['survey']
w2 = model['generation']
print(type(w1))
ans1 = dot(matutils.unitvec(w1), matutils.unitvec(w2))
print(ans1)

ans2 = model.similarity('survey', 'generation')
print(ans2)

<class 'numpy.ndarray'>
0.34138656
0.34138656


In [8]:
import torch
import torch.nn.functional as F

# Assuming 'model' is your pre-loaded word embedding model
x = torch.tensor(model['king'])
y = torch.tensor(model['queen'])
# z = king - man + woman
z = x - torch.tensor(model['man']) + torch.tensor(model['woman'])

# Calculating distances and similarities
distance_king_queen = torch.norm(x - y).item()
cosine_sim_king_queen = F.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0)).item()
distance_king_z = torch.norm(x - z).item()
cosine_sim_king_z = F.cosine_similarity(x.unsqueeze(0), z.unsqueeze(0)).item()

print("Distance between king and queen: ", distance_king_queen)
print("Cosine similarity between king and queen: ", cosine_sim_king_queen)
print("New Distance between king and z: ", distance_king_z)
print("New Cosine similarity between king and z: ", cosine_sim_king_z)


Distance between king and queen:  0.7606731653213501
Cosine similarity between king and queen:  0.7704245448112488
New Distance between king and z:  0.6913657188415527
New Cosine similarity between king and z:  0.8185327053070068


In [9]:
print(type(x))

<class 'torch.Tensor'>


In [9]:
# Load test data

# Load into dataframe
df = pd.read_csv('./SimLex-999/SimLex-999.txt', sep='\t')
print(df.head())

   word1        word2 POS  SimLex999  conc(w1)  conc(w2)  concQ  Assoc(USF)  \
0    old          new   A       1.58      2.72      2.81      2        7.25   
1  smart  intelligent   A       9.20      1.75      2.46      1        7.11   
2   hard    difficult   A       8.77      3.76      2.21      2        5.94   
3  happy     cheerful   A       9.55      2.56      2.34      1        5.85   
4   hard         easy   A       0.95      3.76      2.07      2        5.82   

   SimAssoc333  SD(SimLex)  
0            1        0.41  
1            1        0.67  
2            1        1.19  
3            1        2.18  
4            1        0.93  


In [ ]:
# Get word embeddings
sample_embedding = get_word_embedding(model ,df['word1'][1])
print(sample_embedding.shape)
sample_embedding = sample_embedding.squeeze()
print(sample_embedding.shape)
print(sample_embedding)

(300,)
(300,)
[-0.098387   -0.040395   -0.063524    0.047001   -0.049376    0.011193
  0.018136   -0.1185      0.0044093   0.095713    0.0051247   0.043689
  0.042151   -0.0049011  -0.064752   -0.064047    0.13787     0.057552
  0.10223    -0.010384    0.10026     0.057823   -0.001169    0.062385
  0.008071   -0.0072606   0.041832   -0.0215      0.080674    0.0083488
 -0.018476   -0.06874     0.0059701  -0.050153    0.067923   -0.031816
  0.097436    0.050502   -0.027785   -0.022924   -0.019449   -0.1975
 -0.010258   -0.032903   -0.04815    -0.095292   -0.0098303   0.028774
  0.092227   -0.012334   -0.026945   -0.066757    0.012096   -0.026446
 -0.009822   -0.014377    0.055541    0.011374   -0.045421    0.018927
 -0.016859   -0.051202    0.065667    0.016558    0.022719    0.026686
  0.027099   -0.010242    0.025326   -0.068804    0.059648   -0.05293
  0.037331   -0.026011   -0.044675    0.073512   -0.004466   -0.061549
 -0.024939   -0.06059    -0.020963   -0.092179    0.07017     0.1

In [10]:
# Check similarity between two words
word1 = df['word1'][1]
word2 = df['word1'][1]
# Use gensim matutils to calculate cosine similarity
w1 = model[word1]
w2 = model[word2]

print(type(w1))

sim = dot(matutils.unitvec(w1), matutils.unitvec(w2))
print(sim)

<class 'numpy.ndarray'>
1.0


In [11]:
# Function to get cosine similarity
def cos_similarity(word1_embedding, word2_embedding):
    ans = dot(matutils.unitvec(word1_embedding), matutils.unitvec(word2_embedding))
    return ans

# Function to get Pearson correlation
def pearson_correlation(word1_embedding, word2_embedding):
    emb1 = np.array(word1_embedding)
    emb2 = np.array(word2_embedding)

    correlation, _ = pearsonr(emb1, emb2)
    return correlation

In [12]:
def test_sim(df, lemmatizer, stemmer):
    cosine_similarity_scores = []
    pearson_correlation_scores = []
    simlex_scores = []
    assoc_scores = []
    for _, row in df.iterrows():
        word1 = row['word1']
        word2 = row['word2']
        form = row['POS']
        form = form.lower()
       
        # Get embeddings
        word1_embedding = get_word_embedding(model , word1).squeeze()
        word2_embedding = get_word_embedding(model ,word2).squeeze()

        # Get cosine similarity
        cosine_similarity_scores.append(cos_similarity(word1_embedding, word2_embedding))
        # cosine_similarity_scores.append(model.similarity(word1,word2))

        # Get pearson correlation
        pearson_correlation_scores.append(pearson_correlation(word1_embedding, word2_embedding))

        # Get simlex score
        simlex_scores.append(row['SimLex999'])

        # Get assoc score
        assoc_scores.append(row['Assoc(USF)'])

    return cosine_similarity_scores, pearson_correlation_scores, simlex_scores, assoc_scores

In [16]:
# Get cosine similarity and pearson correlation scores
lemmatizer = WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
cosine_similarity_scores, pearson_correlation_scores, simlex_scores, assoc_scores = test_sim(df, lemmatizer, stemmer)

In [16]:
# Check cosine similarity and pearson correlation scores
print(type(cosine_similarity_scores))
print(type(pearson_correlation_scores))
print(type(simlex_scores))
print(type(assoc_scores))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


### Initial Spearman

In [19]:
# Funtcion to get spearman correlation using cosine similarity scores
def spearman_correlation(cosine_similarity_scores, simlex_scores):
    # Scale cosine similarity scores to 0-10
    cosine_similarity_scores = np.array(cosine_similarity_scores)
    cosine_similarity_scores = (1+cosine_similarity_scores)*5
    # cosine_similarity_scores *= 10
    simlex_scores = np.array(simlex_scores)

    correlation, _ = spearmanr(cosine_similarity_scores, simlex_scores)
    return correlation

In [18]:
# Print the initial spearman correlation
spearman_value_sim = spearman_correlation(cosine_similarity_scores, simlex_scores)
spearman_value_assoc = spearman_correlation(cosine_similarity_scores, assoc_scores)
print("Initial Spearman correlation Sim: ", spearman_value_sim)
print("Initial Spearman correlation Assoc: ", spearman_value_assoc)

Initial Spearman correlation Sim:  0.44090807476822663
Initial Spearman correlation Assoc:  0.43975997475619183


In [13]:
# Function to get df for verbs, adjectives and nouns separately
def get_pos_df(df):
    verb_df = df[df['POS'] == 'V']
    adj_df = df[df['POS'] == 'A']
    noun_df = df[df['POS'] == 'N']

    return verb_df, adj_df, noun_df

In [14]:
# Get dfs for verbs, adjectives and nouns
verb_df, adj_df, noun_df = get_pos_df(df)

In [17]:
# Get cosine similarity and pearson correlation scores for verbs, adjectives and nouns
verb_cosine_similarity_scores, verb_pearson_correlation_scores, verb_simlex_scores, verb_assoc_scores = test_sim(verb_df, lemmatizer, stemmer)
adj_cosine_similarity_scores, adj_pearson_correlation_scores, adj_simlex_scores, adj_assoc_scores = test_sim(adj_df, lemmatizer, stemmer)
noun_cosine_similarity_scores, noun_pearson_correlation_scores, noun_simlex_scores, noun_assoc_scores = test_sim(noun_df, lemmatizer, stemmer)

In [20]:
# Get spearman correlation for verbs, adjectives and nouns
verb_spearman_value_sim = spearman_correlation(verb_cosine_similarity_scores, verb_simlex_scores)
adj_spearman_value_sim = spearman_correlation(adj_cosine_similarity_scores, adj_simlex_scores)
noun_spearman_value_sim = spearman_correlation(noun_cosine_similarity_scores, noun_simlex_scores)

# Print spearman correlation for verbs, adjectives and nouns
print("Spearman correlation Sim for verbs: ", verb_spearman_value_sim)
print("Spearman correlation Sim for adjectives: ", adj_spearman_value_sim)
print("Spearman correlation Sim for nouns: ", noun_spearman_value_sim)

Spearman correlation Sim for verbs:  0.25777445566511625
Spearman correlation Sim for adjectives:  0.5781412840320306
Spearman correlation Sim for nouns:  0.495787237895295


In [19]:
# Make a dataframe of cosine similarity scores and pearson correlation scores along with Simlex-999 scores and Assoc(USF)
simlex_scores = df['SimLex999']
assoc_scores = df['Assoc(USF)']
cosine_similarity_scores = np.array(cosine_similarity_scores)
pearson_correlation_scores = np.array(pearson_correlation_scores)
simlex_scores = np.array(simlex_scores)
assoc_scores = np.array(assoc_scores)
# print(cosine_similarity_scores.shape)
# print(pearson_correlation_scores.shape)

# Make a dataframe along with word1, word2, POS, SimLex-999 scores, Assoc(USF), cosine similarity scores and pearson correlation scores
datat = {'word1': df['word1'], 'word2': df['word2'], 'POS': df['POS'], 'SimLex999': simlex_scores, 'Assoc(USF)': assoc_scores, 'Cosine Similarity': cosine_similarity_scores, 'Pearson Correlation': pearson_correlation_scores}
ndf = pd.DataFrame(data=datat)
print(ndf.head())

   word1        word2 POS  SimLex999  Assoc(USF)  Cosine Similarity  \
0    old          new   A       1.58        7.25           0.733461   
1  smart  intelligent   A       9.20        7.11           0.772989   
2   hard    difficult   A       8.77        5.94           0.729391   
3  happy     cheerful   A       9.55        5.85           0.684763   
4   hard         easy   A       0.95        5.82           0.676172   

   Pearson Correlation  
0             0.732009  
1             0.772902  
2             0.729263  
3             0.685333  
4             0.676259  


In [20]:
# Print df
print(ndf)

      word1        word2 POS  SimLex999  Assoc(USF)  Cosine Similarity  \
0       old          new   A       1.58        7.25           0.733461   
1     smart  intelligent   A       9.20        7.11           0.772989   
2      hard    difficult   A       8.77        5.94           0.729391   
3     happy     cheerful   A       9.55        5.85           0.684763   
4      hard         easy   A       0.95        5.82           0.676172   
..      ...          ...  ..        ...         ...                ...   
994    join      acquire   V       2.85        0.00           0.452802   
995    send       attend   V       1.67        0.00           0.536103   
996  gather       attend   V       4.80        0.00           0.552808   
997  absorb     withdraw   V       2.97        0.00           0.447016   
998  attend       arrive   V       6.08        0.00           0.559992   

     Pearson Correlation  
0               0.732009  
1               0.772902  
2               0.729263  
3  

### Build the model

In [21]:
# Create Dataset
def create_dataset(df, model):
    # Create a list of tuples
    emb1 = []
    emb2 = []
    simlex_scores = []
    assoc_scores = []

    for _, row in df.iterrows():
        word1 = row['word1']
        word2 = row['word2']
        emb1.append(torch.tensor(get_word_embedding(model ,word1)))
        emb2.append(torch.tensor(get_word_embedding(model, word2)))
        simlex_scores.append(row['SimLex999'])
        assoc_scores.append(row['Assoc(USF)'])

    # print(emb1[0].shape)
    emb1_stack = torch.stack(emb1)
    emb2_stack = torch.stack(emb2)

    return emb1_stack, emb2_stack, torch.tensor(simlex_scores, dtype=torch.float), torch.tensor(assoc_scores, dtype=torch.float)

In [22]:
# Call create_dataset
train_df, test_df = train_test_split(ndf, test_size=0.1, random_state=42)
train_emb1, train_emb2, train_simlex_scores, train_assoc_scores = create_dataset(train_df, model)
test_emb1, test_emb2, test_simlex_scores, test_assoc_scores = create_dataset(test_df, model)

In [23]:
# check train_emb1
print(train_emb1.shape)
print(train_simlex_scores.shape)

torch.Size([899, 300])
torch.Size([899])


In [24]:
# Creare TensorDataset
train_dataset = torch.utils.data.TensorDataset(train_emb1, train_emb2, train_simlex_scores, train_assoc_scores)
test_dataset = torch.utils.data.TensorDataset(test_emb1, test_emb2, test_simlex_scores, test_assoc_scores)

In [25]:
# Create DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [26]:
# Class that takes CBOW embeddings, and outputs similarity scores: loss is MSE between predicted similarity scores and actual similarity scores(Simlex-999)
class RegressionModel(nn.Module):
    def __init__(self, embedding_dim):
        super(RegressionModel, self).__init__()
        self.linear1 = nn.Linear(2*embedding_dim, 50)
        self.linear2 = nn.Linear(50, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, emb1, emb2):
        # emb1 = emb1.squeeze()
        # emb2 = emb2.squeeze()
        emb = torch.cat((emb1, emb2), dim=1)

        out = self.linear1(emb)
        out = F.relu(out)
        out = self.dropout(out)
        out = self.linear2(out)

        # Project the output between 0 and 10
        out = torch.sigmoid(out)
        out = out*10
        return out

In [27]:
# Model Parameters
embedding_dim = 300
learning_rate = 0.001
num_epochs = 20
batch_size = 32

# Initialize model
model = RegressionModel(embedding_dim).to(device)
# Define loss function
criterion = nn.MSELoss()
# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.01) # weight_decay is L2 regularization

In [ ]:
# Function to train model
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    train_loss = 0.0
    for epoch in range(num_epochs):
        for emb1, emb2, simlex_scores, assoc_scores in train_loader:
            emb1 = emb1.to(device)
            emb2 = emb2.to(device)
            simlex_scores = simlex_scores.to(device)
            assoc_scores = assoc_scores.to(device)
            # Forward pass
            outputs = model(emb1, emb2)

            simlex_scores = simlex_scores.unsqueeze(1)
            # print(outputs[0])
            loss = criterion(outputs, simlex_scores)
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        print("Epoch: {}, Train_Loss: {}".format(epoch+1, train_loss))

In [ ]:
# Train model
train(model, train_loader, criterion, optimizer, num_epochs)

Epoch: 1, Train_Loss: 6.877282257737784
Epoch: 2, Train_Loss: 6.677323201465267
Epoch: 3, Train_Loss: 6.511966030459557
Epoch: 4, Train_Loss: 6.238854689182254
Epoch: 5, Train_Loss: 6.121467920650745
Epoch: 6, Train_Loss: 5.827177930580805
Epoch: 7, Train_Loss: 5.6249357088101055
Epoch: 8, Train_Loss: 5.694390595725529
Epoch: 9, Train_Loss: 5.340702848214232
Epoch: 10, Train_Loss: 5.307537023931341
Epoch: 11, Train_Loss: 5.2304604116770586
Epoch: 12, Train_Loss: 4.971509925468533
Epoch: 13, Train_Loss: 4.92602253754741
Epoch: 14, Train_Loss: 4.828319475842552
Epoch: 15, Train_Loss: 4.715968417223603
Epoch: 16, Train_Loss: 4.567144403755539
Epoch: 17, Train_Loss: 4.380131288160533
Epoch: 18, Train_Loss: 4.335268984025568
Epoch: 19, Train_Loss: 4.150991686794464
Epoch: 20, Train_Loss: 4.055382589109098


In [ ]:
# Function to test model, Calculate test loss and Spearman correlation
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    true_simlex_scores = []
    pred_simlex_scores = []
    for emb1, emb2, simlex_scores, assoc_scores in test_loader:
        emb1 = emb1.to(device)
        emb2 = emb2.to(device)
        simlex_scores = simlex_scores.to(device)
        assoc_scores = assoc_scores.to(device)
        # Forward pass
        outputs = model(emb1, emb2)
        simlex_scores = simlex_scores.unsqueeze(1)
        loss = criterion(outputs, simlex_scores)
        test_loss += loss.item()

        # Get true labels and predicted labels
        true_simlex_scores.extend(simlex_scores.cpu().detach().numpy().tolist())
        pred_simlex_scores.extend(outputs.cpu().detach().numpy().tolist())

    test_loss /= len(test_loader)
    print("Test_Loss: {}".format(test_loss))
    # Calculate Spearman correlation
    # print("True Simlex scores: ", true_simlex_scores)
    # print("Predicted Simlex scores: ", pred_simlex_scores)

    true_simlex_scores = np.array(true_simlex_scores)
    pred_simlex_scores = np.array(pred_simlex_scores)
    spear = spearmanr(true_simlex_scores, pred_simlex_scores)
    print("Spearman correlation: {}".format(spear[0]))

In [ ]:
# Test model
test(model, test_loader, criterion)

Test_Loss: 6.341392397880554
Spearman correlation: 0.2820332028297094
